In [2]:
import numpy as np, random, operator, pandas as pd

import networkx as nx

In [3]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance

    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"


In [4]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0

    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness


In [5]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [6]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

In [7]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)


In [8]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()

    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [9]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [10]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []

    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])

    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child


In [11]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])

    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [12]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))

            city1 = individual[swapped]
            city2 = individual[swapWith]

            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [13]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []

    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [14]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration


In [15]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations,rewiring):
    pop = initialPopulation(popSize, population)
    #initial watts and strogatz network
    G = nx.generators.random_graphs.watts_strogatz_graph(popSize,3,rewiring)

    scores = []

    for l in range(0,generations):
        gen_scores=[]
        for j in range(0, popSize):
            candidato = pop[j]
            vecinos=[]
            for k in G[j]:
                vecinos.append(k)

            if(len(vecinos)>0):
                #Select best neighbour
                mejor_vecino = 0
                puntaje_mejor_vecino = 100000000

                for k in G[j]:
                    fitness_vecino = 1/Fitness(pop[k]).routeFitness()
                    if(fitness_vecino<puntaje_mejor_vecino):
                        puntaje_mejor_vecino = fitness_vecino
                        mejor_vecino = k

                if(puntaje_mejor_vecino<100000000):
                    pareja = pop[mejor_vecino]
                    child = breed(candidato, pareja)
                    child = mutate(child,mutationRate)

                    score_individuo  = 1/Fitness(candidato).routeFitness()
                    score_child = 1/Fitness(child).routeFitness()

                    if(score_child < score_individuo):
                        pop[j] =  child
                        G.add_edge(j,mejor_vecino)
                        gen_scores.append(score_child)
                    else:
                        gen_scores.append(score_individuo)

        for j in range(0, popSize):
            aristas = G.adj[j]

            vecinos=[]
            for k in aristas:
                vecinos.append(k)

            for i in range(0,len(vecinos)):
                if random.random() < rewiring:
                    random_index = int(random.random()*100)
                    G.remove_edge(j, vecinos[i])
                    G.add_edge(j,random_index)

        gen_best = min(gen_scores)
        scores.append(gen_best)

    return scores

In [17]:
%%time
cityList = []

#25 cities
for i in range(0,5):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.327136632
382.32

In [18]:
%%time
cityList = []

#25 cities
for i in range(0,50):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
3897.608636996
3820.446629802
3738.585419903
3666.676886934
3598.591136379
3514.113287305
3462.938249441
3422.042310070
3380.328806606
3333.129431143
3293.472242867
3256.204568518
3216.259389940
3180.415506749
3141.248052821
3113.934939982
3089.069204329
3069.848044953
3042.679135714
3018.384370508
2990.109876609
2959.509332742
2938.146048138
2919.880046452
2898.215699112
2877.747317479
2853.847646185
2836.540646463
2822.378687551
2800.660992712
2781.571580949
2765.070689560
2756.819834688
2733.529640114
2711.972046764
2695.281056589
2688.121357107
2667.256872002
2657.428736450
2645.959757367
2635.759142298
2626.798570159
2618.107481559
2608.377792925
2588.766836034
257

In [19]:
%%time
cityList = []

#25 cities
for i in range(0,500):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
48226.260374592
47916.103883742
47639.809569399
47284.233720192
47020.865226340
46809.653577672
46586.535055389
46378.610792162
46200.248487071
46019.186818392
45858.917119351
45714.946353377
45547.158646452
45429.623279632
45316.337814828
45193.894510445
45079.110515894
44944.225250786
44811.161698095
44708.485463404
44625.125016150
44506.303249242
44455.230354145
44345.465683953
44261.811666960
44210.082991194
44126.992861055
44023.871627131
43925.623785826
43856.066052072
43769.853611021
43671.377817069
43611.612673799
43525.186145537
43478.596654311
43431.092947798
43369.031265123
43305.993962466
43254.391791296
43174.137918624
43101.925960634
43029.866082051
42988.

In [16]:
%%time
cityList = []

#25 cities
for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
2045.068358674
1978.495984036
1925.926497862
1872.055511678
1818.374315724
1780.984240718
1737.141017463
1701.977166952
1667.003604578
1640.680732809
1614.730603502
1598.314108881
1565.410088760
1534.889010644
1518.351115367
1497.639728555
1478.043198438
1464.885542731
1442.154406866
1422.908005321
1410.997271562
1389.877554941
1378.281907034
1361.539086396
1354.579652594
1343.798006952
1334.855266725
1319.772902641
1311.110515648
1298.525746499
1292.489951072
1281.785341363
1271.584915358
1262.184046599
1250.452433006
1244.893165520
1237.304037885
1232.976012583
1228.362830467
1218.733756599
1217.588949816
1212.030329721
1206.224678764
1196.804233589
1190.344644404
118

In [17]:
%%time
cityList = []

#25 cities
for i in range(0,1000):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
100222.148924941
99763.016021433
99291.482419726
98889.879897013
98550.590771198
98193.221158799
97876.993931648
97620.412081140
97271.132256796
96977.133986894
96721.698819923
96487.014320168
96308.208880205
96127.795303696
95965.214098766
95800.339852911
95549.982050540
95412.764406814
95282.847524700
95072.954006838
94909.800843157
94809.720942924
94674.900435716
94528.222334894
94379.849857443
94266.718221493
94206.725547907
94022.529990175
93926.920764941
93801.033166474
93705.044027562
93588.101782933
93470.385651175
93366.063876018
93234.754351407
93146.344715692
93066.497758036
92981.967886703
92880.685966053
92771.770754809
92700.861112235
92617.658150575
92561